In [ ]:
# default_exp main

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Loading

> Module to load the roster table.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
from pathlib import Path
import re
import pandas as pd
from typing import Any
from datetime import timedelta
from roster2ical.roster import Shift, ShiftProperties, Roster

In [ ]:
data_path = Path.cwd() / "data"
# roster_path = data_path / "202201_example_roster.xlsx"
roster_path = data_path / "private" / "202202_ZIB_Feb.xlsx"

day_pattern = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
date_pattern = re.compile(r"\d{2}")


def daycol(col):
    return day_pattern.search(col)


# TODO: Should use daycols for variable length months
df = pd.read_excel(roster_path, skiprows=0, header=1, index_col=0)
roster_df = (
    df.dropna().rename_axis("Name").loc[:, [col for col in df.columns if daycol(col)]]
)
roster_df

In [ ]:
roster_df = roster_df.drop(columns=["DI 01.1"], errors="ignore")
roster_df.iloc[:, 5:]

In [ ]:
abbr2shiftproperties = {
    "(/)": None,
    "2M": None,
    "3g": None,
    "RTH": None,
    "NEF": None,
    "NAW 24": None,
    "V": None,
    "F0": ShiftProperties(
        name="Früh0", starting_hour=timedelta(hours=8), duration=timedelta(hours=10)
    ),
    "F1": ShiftProperties(
        name="Früh1", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "F2": ShiftProperties(
        name="Früh2", starting_hour=timedelta(hours=8), duration=timedelta(hours=12)
    ),
    "FB": ShiftProperties(
        name="Fortbildung",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
    "N1": ShiftProperties(
        name="Nacht1", starting_hour=timedelta(hours=22), duration=timedelta(hours=10)
    ),
    "N2": ShiftProperties(
        name="Nacht2", starting_hour=timedelta(hours=20), duration=timedelta(hours=12)
    ),
    "S1": ShiftProperties(
        name="Spät1", starting_hour=timedelta(hours=15), duration=timedelta(hours=8)
    ),
    "U": ShiftProperties(
        name="Urlaub", starting_hour=timedelta(hours=8), duration=timedelta(hours=8)
    ),
    "ZA": ShiftProperties(
        name="Zeitausgleich",
        starting_hour=timedelta(hours=8),
        duration=timedelta(hours=8),
    ),
}


In [ ]:
name = ""
r = Roster.from_dict(roster_df.loc[name].to_dict(), mapper=abbr2shiftproperties)

In [ ]:
r

In [ ]:
for name in roster_df.index:
    try:
        r = Roster.from_dict(roster_df.loc[name].to_dict(), mapper=abbr2shiftproperties)
    except:
        print(f"Could not create roster for {name}")
        continue
    filename = f"{name.replace(',', '_').replace(' ', '')}.ics"
    with (data_path / "output" / filename).open("w") as f:
        f.write(str(r.to_ics()))
